In [1]:
import os
import cv2
import tqdm
import pickle
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from utils.dataset_utils import *
from collections import defaultdict
from deep_sort_realtime.deepsort_tracker import DeepSort

In [2]:
video = "video_8min"
num_frames=1
total_detections=33

In [3]:
raw_video_folder = r'..\data\raw\videos'
video_path = raw_video_folder + "\\" + video + ".mp4"

yolo_path = r'..\models\costumized_yolo\costumized_yolo\costumized_yolo.pt'
processed_video_folder = rf'..\data\processed\{video}'

In [4]:
model = YOLO(yolo_path)
tracker = DeepSort(max_age=30)
cap = cv2.VideoCapture(video_path)
total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

c:\Users\janni\anaconda3\envs\GAIL\lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [5]:
# total_frames
os.makedirs(os.path.join(processed_video_folder, "total_frames"), exist_ok=True)
tf_path = os.path.join(processed_video_folder, "total_frames", "total_frames.pkl")

if os.path.exists(tf_path):
    with open(tf_path, "rb") as f:
        total_frames = pickle.load(f)
else:
    total_frames = []
    for frame in tqdm.tqdm(range(total_frame_count), desc="Processing frames"):
        frame_records = process_frame(cap, model, tracker, frame)
        total_frames.extend(frame_records)
    cap.release()
    with open(tf_path, "wb") as f:
        pickle.dump(total_frames, f)

In [6]:
# filtered_frames
os.makedirs(os.path.join(processed_video_folder, "filtered_frames"), exist_ok=True)
ff_path = os.path.join(processed_video_folder, "filtered_frames", "filtered_frames.pkl")
ms_path = os.path.join(processed_video_folder, "filtered_frames", "max_speed.pkl")

if os.path.exists(ff_path) and os.path.exists(ms_path):
    with open(ff_path, "rb") as f:
        filtered_frames = pickle.load(f)

    with open(ms_path, "rb") as f:
        max_speed = pickle.load(f)
else:
    filtered_frames, max_speed = filter_frames(total_frames)
    with open(ff_path, "wb") as f:
        pickle.dump(filtered_frames, f)

In [ ]:
# full_track_windows
os.makedirs(os.path.join(processed_video_folder, "full_track_windows"), exist_ok=True)
ftw_path = os.path.join(processed_video_folder, "full_track_windows", f"full_track_windows_{total_detections}.pkl")
vw_path = os.path.join(processed_video_folder, "full_track_windows", f"valid_windows_{total_detections}.pkl")

if os.path.exists(ftw_path) and os.path.exists(vw_path):
    with open(ftw_path, "rb") as f:
        full_track_windows = pickle.load(f)

    with open(vw_path, "rb") as f:
        valid_windows = pickle.load(f)
else:
    full_track_windows, valid_windows = find_valid_windows(filtered_frames, num_frames=num_frames, total_detections=total_detections)
    with open(ftw_path, "wb") as f:
        pickle.dump(full_track_windows, f)

    with open(vw_path, "wb") as f:
        pickle.dump(valid_windows, f)

print(f"Found {len(valid_windows)} windows with {total_detections} continuous detections.")

Found 742 windows with 33 continuous detections.


In [10]:
# get expert tensors
os.makedirs(os.path.join(processed_video_folder, "expert_tensors"), exist_ok=True)
pred_et_path = os.path.join(processed_video_folder, "expert_tensors", f"pred_tensors_{num_frames}_{total_detections}.pkl")
prey_et_path = os.path.join(processed_video_folder, "expert_tensors", f"prey_tensors_{num_frames}_{total_detections}.pkl")

if os.path.exists(pred_et_path) and os.path.exists(prey_et_path):
    with open(pred_et_path, "rb") as f:
        pred_tensors = pickle.load(f)

    with open(prey_et_path, "rb") as f:
        prey_tensors = pickle.load(f)
else:
    pred_tensors, prey_tensors = get_expert_tensors(full_track_windows, valid_windows, width, height, max_speed, window_size=num_frames)
    with open(pred_et_path, "wb") as f:
        pickle.dump(pred_tensors, f)

    with open(prey_et_path, "wb") as f:
        pickle.dump(prey_tensors, f)

print("Pred Tensors Shape:", pred_tensors.shape)
print("Prey Tensors Shape:", prey_tensors.shape)

Pred Tensors Shape: torch.Size([742, 1, 1, 32, 5])
Prey Tensors Shape: torch.Size([742, 1, 32, 32, 5])
